![Helper Logo](https://s3.amazonaws.com/public-rafa/impala-helper-logo-t.png)

# Introducción a Impala Helper

**Impala Helper** Es una Librería que permite interactuar con Impala desde Python, para facilitar la calendarización y automatización de procesos que involucren Consultas SQL al Aljibe de datos.

## Prerrequisitos

*  PyODBC (Instalar con Artifactory)
*  Pysftp (Instalar con Artifactory)
*  paramiko (Instalar con Artifactory)

*  Driver de ODBC de Impala (Descargar del [sitio de Cloudera](https://www.cloudera.com/downloads/connectors/impala/odbc/2-6-2.html))
*  Pandas (Viene con Anaconda preinstalado)
*  Certificado Digital Para conectarse a Impala (pedirlo a DiCAGI!)
*  PyArrow
* **Si ya tienes HewNoMore ya tienes algunos de los prerrequisitos**


## Funcionalidades

*  **getQueries**( _rutaArchivo , params_ ) : Trae la lista de queries de un archivo, si se envían parámetros los reemplaza al traer la lista de consultas.

*  **executeFile**(_rutaArchivo , params_ ) : Ejecuta los queries de un archivo. Si le mandas parámetros lo reemplaza antes de ejecutarlos.

*  **execute**(_query , params_ ) : Ejecuta un query en específico.  especifico para lanzar consultas sin devolución de resultados.

*  **toCSV** ( _query , rutaDestino , params_) : Ejecuta un query y manda los resultados a CSV a la ruta que le mandes. Si le mandas parámetros lo reemplaza antes de ejecutarlo.

*  **fromCSV**(_rutaArchivo , nombreTabla , serverOpts_) : Toma un archivo CSV , trata de estimar cuales son los tipos de datos y crea una tabla nueva en impala.  

*  **fromPandasDF**(_dataFrame , nombreTabla , serverOpts_) : Toma un dataframe de Pandas , trata de estimar cuales son los tipos de datos y crea una tabla nueva en impala.  

*  **getRows**( _query , params_ ) : Trae los resultados de un Query en una lista de python.  Lo ideal es que esta función solo se utilice para resultados limitados.  Por ejemplo no hacerlo para traerse 1 millón de registros.

*  **getDataFrame**( _query , params_ ) : Trae los resultados de un Query a un DataFrame de Pandas.  Lo ideal es que esta función solo se utilice para resultados limitados.  Por ejemplo no hacerlo para traerse 1 millón de registros.

*  **getDataFrame**( _query , params_ ) : Trae los resultados de un Query a un DataFrame de Pandas.  Lo ideal es que esta función solo se utilice para resultados limitados.  Por ejemplo no hacerlo para traerse 1 millón de registros.

*  **tableSize**( _tabla , sizeFmt = "B"_ ):  Devuelve el tamaño (almacenamiento físico) de una tabla de impala.  Por defecto el valor esta en bytes.  Se puede establecer los siguientes formatos de tamaño: "YB","ZB","EB","PB","TB","GB","MB","KB", "B".

*  **muestraAleatoria**( _tabla , pct = 0.01_ ):  Genera una muestra aleatoria de una tabla , por defecto es una muestra del 1% de los datos.

*  **muestraEstratificada**( _tabla , estratos , pct = 0.01_ ):  Genera una muestra aleatoria estratificada de una tabla , se envia tambien las columnas que combinan los estratos.  por defecto se tiene una muestra del 1%.

*  **oneHot**( _table , colsOHE_) : Añade columnas por medio de _**One Hot Encoding (OHE)**_ a una tabla.  Recibe la tabla como parámetro y una lista de columnas a las cuales hacer OHE. La función hace un distinct de los valores de cada columna y genera una nueva columna por cada valor.  La función retorna un diccionario donde cada id (columna nueva) identifica a que valor corresponde.   

*  **transpose**( _tabla , colsGroup , colsTrans , colsSum_) : Una semitransposición de una tabla, donde se definen columnas de agrupación, las columnas a transponer y que valor se debe poner en la suma de la transposición.

*  **enmascarar**( _tabla , variables , saltI = None , saltT = None_) : Enmascaramiento de variables en una tabla (bajo función de Hash de 512 bits). Usa funciones propias de impala para generar variables enmascaradas. Puede utilizar sales enviadas o generadas automáticamente.

*  **enmascararLong**( _tabla , variables , tabla_original = None_) : Enmascaramiento de variables en una tabla, tratando de evitar cambios en longitud de campos.  Usa 'tablas de verdad' con el valor original y el generado para cada campo.

*  **estabVariables**( _tabla , groupCols , varbl = None , exclude = None_) : Esta funcionalidad permite hacer un análisis de la estabilidad de variables en términos de algún(os) campo(s) de la tabla y genera un análisis de la distribución de las variables.

*  **useDataBase**( _nombreDB_ ) : Por si se quiere cambiar la base de datos por defecto en el camino.

*  **close()**  :  Al finalizar el programa se debe cerrar la conexión.

## Modo de Uso

A continuación se describe paso a paso como utilizar la librería.  

#### Acceso a GIT Analítico

*  Debes tener un usuario en el Git Analítico.  Se puede crear Directamente sin pedir permiso [aqui](http://sbmdeqpc04/)!
*  Pedir acceso al repositorio **Herramientas_Comunidad**.  Se puede solicitar a cualquier miembro de DiCAGI.
*  Hacer un clone del repositorio en tu carpeta de git:
> git clone http://sbmdeqpc04/rlarios/Herramientas_Comunidad.git
*  Ya estas listo para comenzar a utilizar Impala Helper! (si tienes los prerrequisitos)

A continuación se muestra el código de como usarlo!

# Inicialización

In [ ]:
import sys 

# Se agrega la ruta donde quedó el Helper al path de direcciones de python
sys.path.append( 'D:/git/Herramientas_Comunidad/aljibe_helper/'  )

#Se agrega la librería
from Impala_Helper import Helper 

# Configuración

Se debe definir un diccionario de Python con las siguientes variables:

### Parámetros Obligatorios
* **connStr**: Cadena de conexión a Impala.  Si tienes configurado el driver en tu equipo solo es necesario poner el nombre de como quedó configurado. por ejemplo, si en el administrador ODBC tienes una conexión configurada como impala-prod, la cadena de conexión sería **DSN=impala-prod**


### Parámetros Opcionales
* **db**:  (str) Base de Datos por defecto.  En general debe ser la base de datos de proceso del área.  Si no se envía por defecto es **default**.
* **refresh**: (int) Indica el número de segundos que espera para refrescar la conexión.  El valor por defecto es **300** segundos, solo cambiar si se tienen procesos largos que pudieran beneficiarse por tener una conexión por mayor tiempo.  Recomendable **no** poner un valor de más de 10x60 segundos (10 minutos)
* **infoLeng**:  (int) Limita el tamaño de los mensajes que muestra a una cantidad limitada.  Por defecto está en **50**.
* **fetchSize**: (int) El tamaño de registros a traer por solicitud.  Esta en un valor por defecto de **10000**.  **Si no sabe como utilizarlo no cambie este parámetro**
* **verbose**: (bool) Indica si imprime cada acción que ejecuta.  valor por defecto es **False**


In [ ]:
cache = {
            "connStr" : "DSN=Impala" ,
            "db" : "proceso_cap_analit_y_gob_de_inf" ,
            "verbose" : True
        }

hp = Helper( cache )

### Inicializar con Log

En el caso que se desee inicializar el Helper y que automáticamente genere un archivo de log (para facilitar procesos de calendarización de procesos), se puede crear el logger (que está en la misma librería de impala helper) y se debe crear un objeto de log:

In [ ]:
from Impala_Helper import Helper 
from logger import logger

cache = { "connStr" : "DSN=Impala"  }

log = logger( pathlog ="/ruta/donde/almacenar/log/" , logName = "nombre_log.log" )
hp = Helper(cache , logger = log)




# FUNCIONALIDADES


# Ejecutar Un archivo sin parámetros

La función recibe la ruta de un archivo .sql el cual ejecutará en el orden que se encuentren las consultas.  las consultas deberán estar separadas por punto y coma ( ; )

In [ ]:
rutaArchivo = 'D:/git/Herramientas_Comunidad/aljibe_helper/queries_ejemplo/sin_params.sql'

hp.executeFile( rutaArchivo )

# Ejecutar Un archivo con parámetros

Uno puede ejecutar archivos con parámetros definidos en el archivo SQL.  Para definir un parámetro este debe estar dentro de llaves.  por ejemplo:

> ... where year = {**anhoEjec**} 

Los parámetros que se envían a Helper son variables que se definen en un diccionario de python.  Estos pueden ser definidos fijos (como esta abajo) o crear una funcion que los calcule automaticamente, como la fecha de Hoy por ejemplo.

In [ ]:
rutaArchivo2 = 'D:/git/Herramientas_Comunidad/aljibe_helper/queries_ejemplo/con_params.sql'

import datetime

# se calcula Hoy - 10 días
hoy = datetime.date.today()
mesA = (hoy - datetime.timedelta(10)).strftime('%Y%m%d')

params = {
    "anhoFijo" : 2018 ,
    "partition" : datetime.date.today().strftime('%Y') ,
    "startDay" : mesA 
}

hp.executeFile( rutaArchivo2 , params )

## Traer Todos los queries de un archivo
En algunos casos nos gustaría traer todas las consultas de un archivo.  Estos queries quedan en una lista de python.  Si a la función se le especifica un diccionario de parámetros como en el ejemplo anterior, la función reemplaza los parámetros en los queries antes de retornarlos.

In [ ]:
params = {
    "partition" : 2018 ,
    "startDay" : 20181001
}

print("\nQueries sin cambiar parámetros:\n")
queries = hp.getQueries( rutaArchivo2  )
print(queries)
print( )

print("\nQueries cambiando parámetros:\n")
queries = hp.getQueries( rutaArchivo2 , params )
# Note que a los queries ya se le cambiaron los parámetros por las variables enviadas.
print(queries)


# Traer una Consulta a CSV

En ocasiones debemos traer resultados al equipo local en formato CSV.  Es posible traer una consulta e incluso usar parámetros en ella. (En este ejemplo no se envían parámetros)

In [ ]:
rutaCSV = "d:/Ejemplo_IH.csv"
query = "select * from hp_debito_base limit 10"

hp.toCSV( query , rutaCSV ) 

# Enviar un archivo CSV a Tabla Impala

En el caso de que queramos subir una tabla de parámetros a veces es necesario subir un archivo CSV como tabla impala.  Esta función internamente ejecuta un proceso de enviar el archivo al servidor y crear una **EXTERNAL TABLE** (el usuario debe tener permiso para crearla).

Para esto la función requiere el parámetro de serverOpts, el cual puede ser de 2 tipos
*  La ruta a un archivo json donde se encuentre los parámetros de user y password
* un diccionario de python con los parámetros:
> serverOpts = { "user" : "tuUsuario" , "password" : "tuContraseña" } 



In [ ]:
rutaCSV = "d:/Ejemplo_IH.csv"
tabla = "tabla_parametros"
serverOpts = { "user" : "tuUsuario" , "password" : "tuContraseña" }

hp.fromCSV( rutaCSV , tabla , serverOpts) 

# Enviar un dataframe de Pandas a Tabla Impala

En el caso de que queramos subir una tabla de parámetros a veces es necesario subir un DataFrame de Pandas como tabla impala. Esta función internamente ejecuta un proceso de enviar el archivo al servidor y crear una **EXTERNAL TABLE** (el usuario debe tener permiso para crearla).

Para esto la función requiere el parámetro de serverOpts, el cual puede ser de 2 tipos
*  La ruta a un archivo json donde se encuentre los parámetros de user y password
* un diccionario de python con los parámetros:
> serverOpts = { "user" : "tuUsuario" , "password" : "tuContraseña" } 


In [ ]:
query = "select * from hp_debito_base limit 5"

df = hp.getDataFrame(query)
tabla = "tabla_parametros_desde_df"
serverOpts = "d:/archivoPass.json"

hp.fromPandasDF( df , tabla , serverOpts) 

# Traer los resultados a una lista de Python
Trae un query a una lista de python y devuelve un header (nombres de columnas separadas por comas) y la lista de python.  También recibe parámetros en caso de ser necesario.

La función retorna dos variables:
*  El header que es un string con las columnas del query (separado por comas).
*  Los resultados que es una lista de python, donde cada posición es una lista con cada registro (cada registr es una lista de campos)

In [ ]:
query = "select * from hp_debito_base limit 100"

header , results = hp.getRows(query)

print("\nColumnas: " + header)
print("\nPrimeras 10 filas:")
for i in range(10):
    print("Fila {0}: {1}".format(i,results[i]))

# Traer los resultados a un DataFrame de Pandas
Trae un query a una dataFrame de pandas.  También recibe parámetros en caso de ser necesario.

In [ ]:
query = "select * from hp_debito_base limit 100"

df = hp.getDataFrame(query)

df

# Consultar el tamaño de una tambla (Almacenamiento físico)

Devuelve el tamaño (almacenamiento físico) de una tabla de impala.  Internamente lee todos los archivos de la tabla y extrae el tamaño individual de cada uno y los suma.  Por defecto el formato esta en **Bytes**.  Se puede establecer los siguientes formatos de tamaño: "YB","ZB","EB","PB","TB","GB","MB","KB", "B". 

In [ ]:
tabla = "resultados_vspc_clientes.master_customer_data"
formatos = ["MB" , "GB" , "TB"]

sizes = [] 
for fmt in formatos:
    tamanho = hp.tableSize( tabla , fmt )    
    sizes.append( ( fmt , round(tamanho , 2) ) )
    
sizes

# Generar una muestra aleatoria de una tabla
Genera una muestra aleatoria de una tabla. El tamaño de la muestra por defecto es del 1% pero se puede cambiar.


La función recibe obligatoriamente el nombre de la tabla y las columnas con las categorías/estratos, recibe un parámetro opcional que es el tamaño de la muestra medido en el porcentaje de registros.  El resultado es una tabla del mismo nombre de la original con el sufijo **\_muestra**

In [ ]:
# Generación de una tabla base para las muestras

query = """
create table tabla_cli stored as parquet as 
    select tipo_cli ,  ciudad_nacim ,  cal_sarlaft
    from resultados_vspc_clientes.master_customer_data 
    where year = 2019 and month = 11 and ingestion_day = 21
"""
hp.execute("drop table if exists tabla_cli")
hp.execute( query )
hp.execute( "compute stats tabla_cli" )
df = hp.getDataFrame( "select count(*) from tabla_cli ")
df

In [ ]:
# Muestra aleatoria del 5%

hp.muestraAleatoria( "tabla_cli" , pct = 0.05  )

df = hp.getDataFrame( "select count(*) from tabla_cli_muestra ")
df

# Generar una muestra aleatoria estratificada de una tabla
Genera una muestra aleatoria de una tabla por estratos o categorías. El tamaño de la muestra por defecto es del 1% pero se puede cambiar.

La función recibe obligatoriamente el nombre de la tabla y las columnas con las categorías/estratos, recibe un parámetro opcional que es el tamaño de la muestra medido en el porcentaje de registros.  El resultado es una tabla del mismo nombre de la original con el sufijo **\_muestra\_est**

In [ ]:
# Muestra aleatoria del 3% usando estratos las columnas de tipo cliente y ciudad
estratos = ["tipo_cli" ,  "ciudad_nacim"]

# Conteo de estratos en la tabla original.
df1 = hp.getDataFrame("select tipo_cli , trim(ciudad_nacim) ciudad , count(*) as cnt from tabla_cli group by tipo_cli , ciudad_nacim")
print(df1)

hp.muestraEstratificada( "tabla_cli" , estratos , pct = 0.03  )

# Conteo de estratos en la tabla de muestra.
df = hp.getDataFrame( "select tipo_cli , trim(ciudad_nacim) ciudad , count(*)  as cnt from tabla_cli_muestra_est group by tipo_cli , ciudad_nacim ")
print(df)

# Generación One Hot Encoding
Genera columnas con One Hot Encodings a una tabla y crea una tabla nueva con las columnas generadas.  La función tiene una variable opcional que llamada drop.  En caso de ser igual a __True__ elimina las columnas Originales que están en el One Hot Encoding.

In [ ]:
# Se genera OHE para tabla hp_debito_base y columnas de Moneda y Bin
rela =  hp.oneHot("hp_debito_base" , [ "pais" ] , drop = True )

print("\nTotal de columnas nuevas: {0}\n".format(len(rela)))

hp.getDataFrame("Select * from hp_debito_base_ohe limit 10" )

# Tabla Dinámica (Transposición)

Genera una nueva tabla con la transposición de una o más columnas, la función recibe el nombre de la tabla , una lista de columnas a transponer y una lista de columnas de valor que deben asignarse a la columna transpuesta.

In [ ]:
df1 = hp.getDataFrame("Select * from myCNAME  limit 100" )

hp.transpose( "myCNAME" , ["tipo_cliente"] , ["control_terceros"] , ["tipo_doc"]) 

df2 = hp.getDataFrame("Select * from myCNAME_trans  limit 100" )

In [ ]:
df1

In [ ]:
df2

# Estabilidad Variables
Esta funcionalidad permite hacer dos cosas:

1. Un análisis de la estabilidad de variables en términos de algún(os) campo(s) de la tabla. Por ejemplo: variables en función de una fecha de análisis.
2. Un análisis de la distribución de las variables.

## Análisis de Estabilidad
El análisis de estabilidad consiste en calcular límites de control sobre la media y el porcentaje de completitud.

### Media 
La media se calcula para todas las variables a analizar, agrupada por la(s) variable(s) de agrupación. Una vez calculada se verifica: para cada variable, en cuántos periodos la variable toma valores por fuera de las siguientes cotas:

\begin{equation*}
l = \mu \pm Z_\alpha * \frac{\sigma}{\sqrt{n}}
\end{equation*}

Donde $\mu$ es la media de la variable en todos los periodos. $Z_\alpha$ corresponde al nivel de confianza (que se calcula con 90%, 95% y 99%). $\sigma$ es la desviación estándar de todos los datos. $n$ el número de valores únicos observados observados en la variable de agrupación.

### Porcentaje de No Completitud

En el caso del procentaje de *missing*, las cotas están dadas por:

\begin{equation*}
l = \bar{p} \pm Z_\alpha \sqrt{\frac{\bar{p} ( 1 - \bar{p}  ) }{n}}
\end{equation*}

En donde $\bar{p}$ es el promedio de toda la población.


## Análisis de Distribución
Este módulo permite consultar para cada variable: el procentaje de completitud, el mínimo, el máximo y los percentiles 1, 10, 25, 50, 75, 90 y 99.


## Ejecución
Para ejecutar el código se debe indicar: (1) la tabla a revisar, (2) la variable que se utilizará para hacer el análisis de estabilidad (i.e. periodo), y (3) las variables sobre las que se hará el análisis ó las variables que deben ser excluídas del análisis (i.e. llaves).

In [ ]:
tabla = "proceso_cap_analit_y_gob_de_inf.rb_trx_dba_6" # Tabla sobre la que se realizará el análisis
grp = ["f_analisis"] # Variable(s) para agrupar, para el análisis de estabilidad
variables = ['num_doc' , 'tipo_doc', 'llave_nombre', 'f_analisis' ] #Variables a excluir del análisis

dfRet , dfRetT, df = hp.estabVariables(tabla , grp ,  exclude = variables )

Notar que en el ejemplo anterior se incluyó la lista de variables a excluir del análisis. Si lo que se quiere es incluir las variables a analizar el comando a utilizar es el siguiente:

In [ ]:
variables_analizar = ["trx_dba_cnt_cta_6vs12", "trx_dba_sld_cta_6vs12", "trx_dba_sld_sg_6vs12", "trx_dba_sld_max_6vs12"]
dfRet , dfRetT, df = hp.estabVariables(tabla , grp ,  variables_analizar )

## Resultados
El helper retorna tres dataframes: `dfRet`, `dfRetT`, y `df`.

### `dfRet`
Este dataframe tiene la información agregada por los valores únicos que toma `grp`. Específicamente, para cada valor indica cuántas variables se salen del intervalo de confianza calculado por los límites de control. El siguiente código de python le puede ayudar a tener una vista resumida de los resultados de este dataframe:

```python
dfRet['tabla'] = dfRet['tabla'].str.replace( tabla + '_', '')
dfRet['periodo'] = dfRet['periodo'].str.replace('f_analisis: ', '')

df2 = dfRet['tabla'].apply(lambda x: pd.Series(x.split('_')))
df2.columns = ['analisis','intervalo','tipo']
dff_p = pd.concat([df2,dfRet], axis=1)
dff_p['n_variables'] = dff_p['variables'].str.count(',') + 1
```

El código anterior genera una tabla de 6 columnas:

1. Análisis: permite identificar si es de completitud o de medias (lcm).
2. Intervalo: puede tomar los valores de 90, 95 y 99.
3. Tipo: se usa para discriminar si el análisis es por media o por mediana. Ignorar el de mediana ya que tiene un error en el cálculo.
4. Periodo: valores que toman las variables de agrupación.
5. n_variables: número de variables que se salen del intervalo.
6. Variables: variables específicas que se salen del intervalo.


### `dfRetT`
Este dataframe puede entenderse como `dfRet` transpuesto. Lo que informa es para cada variable en cuantos valores únicos de `grp`, las medidas calculadas se salen del límite de control. El siguiente código de python le puede ayudar a resumir:

```python
deRetT['tabla'] = deRetT['tabla'].str.replace( tabla + '_', '')

df2 = deRetT['tabla'].apply(lambda x: pd.Series(x.split('_')))
df2.columns = ['analisis','intervalo','tipo']
dff_v = pd.concat([df2,deRetT], axis=1)
dff_v['n_periodos'] = dff_v['periodos'].str.count('--') + 1
```
El código anterior genera una tabla de 6 columnas:

1. Análisis: permite identificar si es de completitud o de medias (lcm).
2. Intervalo: puede tomar los valores de 90, 95 y 99.
3. Tipo: se usa para discriminar si el análisis es por media o por mediana. Ignorar el de mediana ya que tiene un error en el cálculo.
4. Periodo: valores que toman las variables de agrupación.
5. n_variables: número de variables que se salen del intervalo.
6. Variables: variables específicas que se salen del intervalo.




### `df`
Este dataframe muestra, para cada variable: el procentaje de completitud, el mínimo, el máximo y los percentiles 1, 10, 25, 50, 75, 90 y 99. Estos cálculos se hacen sin discriminar por las variables de agrupación `grp`.